In [11]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [12]:
using JuMP, GLPK
using DataFrames
using CSV

In [13]:
# Load the data
scenarios_200 = CSV.read("../data/scenarios_200.csv", DataFrame)

# create a dictonary with 200 dataframes for each scenario
scenarios = Dict()
for i in 1:250
    df_helper = DataFrame(scenarios_200[:,3*i-2:3*i])
    #set column names
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    scenarios[i] = df_helper
end

In [14]:
scenarios[1]

Row,price,wind power,grid_excess
,Float64,Float64,Int64
1,122.83,151.992,1
2,132.91,143.459,0
3,132.9,142.587,0
4,125.03,146.618,0
5,120.07,150.136,1
6,139.83,151.102,1
7,164.23,145.927,1
8,186.66,143.261,0
9,186.95,142.279,0


In [15]:
scenarios[2]

Row,price,wind power,grid_excess
,Float64,Float64,Int64
1,2.01,22.3372,1
2,1.38,18.6099,0
3,0.09,15.9499,0
4,0.08,13.4696,0
5,0.05,9.2429,1
6,0.0,5.90827,1
7,-0.38,4.51948,1
8,0.0,2.73391,0
9,0.0,1.1039,0


In [16]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[])

Row,hour,p_DA
,Int64,Float64


In [17]:
W = 250
hours = 24

24

In [18]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:hours])
@variable(model, delta[1:W,1:hours])
@variable(model, delta_up[1:W,1:hours])
@variable(model, delta_down[1:W,1:hours])

# Define the objective function
@objective(model, Max, sum(1/W*(
        scenarios[i][hour,"price"] * p_DA[hour]
     + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
      - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)
      ) for i in 1:W, hour in 1:hours))

# Define the constraints
@constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
@constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)



# Solve the optimization problem
optimize!(model)


#println(model)

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found")
end

Optimal solution found
Objective value: 130635.8925899926


In [19]:
println("Objective value: ", objective_value(model))
println("p_DA: ", value.(p_DA))
println("delta: ", value.(delta))
println("delta_up: ", value.(delta_up))
println("delta_down: ", value.(delta_down))

Objective value: 130635.8925899926
p_DA: [51.54661290322581, 14.389693451612905, 51.14903225806452, 45.99032258064516, 43.07419354838709, 43.34193548387096, 145.9269354516129, 13.34106441935484, 1.1038967741935486, 146.94499998387096, 22.81691932258065, 0.7419306451612904, 142.91516125806453, 40.880483870967744, 135.9887096612903, 40.13322579032258, 146.0248386935484, 149.60145158064518, 16.439596645161288, 152.74596772580645, 4.493516129032258, 60.77661290322581, 6.808774177419355, 149.57870966129033]
delta: [100.44580641935484 129.06950009677416 91.43774193548387 100.6280645 107.06177414516128 107.75967735483869 0.0 129.9200645967742 141.1748128548387 -4.729677467741908 118.76179030645162 137.57581127419354 -5.579999983870977 95.92596772580646 -2.991612919354793 89.39596767741938 -24.194838741935484 -34.47548390322582 92.0618549032258 -49.20499998387098 93.68874191935484 31.38935477419355 77.84396772580645 -71.5091935483871; -29.209371064516127 4.2201774193548385 -35.19909680645162 -